In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'imutils'

In [2]:
dataset=r'C:\Users\omdha\jupyter nb\Mask Dataset'
imagePaths=list(paths.list_images(dataset))

In [3]:
imagePaths

['C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\0.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\1.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\10.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\100.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\101.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\102.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\104.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\105.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\106.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\107.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\108.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\109.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\11.jpg',
 'C:\\Users\\omdha\\jupyter nb\\Mask Dataset\\without_mask\\110.jpg',
 'C:\\Users\\omdha\\jupyte

In [4]:
data=[]
labels=[]

for i in imagePaths:
    label=i.split(os.path.sep)[-2]
    labels.append(label)
    image=load_img(i,target_size=(224,224))
    image=img_to_array(image)
    image=preprocess_input(image)
    data.append(image)

In [5]:
data=np.array(data,dtype='float32')
labels=np.array(labels)

In [6]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [ ]:
labels

In [7]:
train_X,test_X,train_Y,test_Y=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=10)

In [8]:
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest')

In [9]:
baseModel=MobileNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [10]:
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name='Flatten')(headModel)
headModel=Dense(128,activation='relu')(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation='softmax')(headModel)

model=Model(inputs=baseModel.input,outputs=headModel)

In [11]:
for layer in baseModel.layers:
    layer.trainable=False

In [12]:
baseModel.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [ ]:
learning_rate=0.001
Epochs=20
BS=12

opt=Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

H=model.fit(
    aug.flow(train_X,train_Y,batch_size=BS),
    steps_per_epoch=len(train_X)//BS,
    validation_data=(test_X,test_Y),
    validation_steps=len(test_X)//BS,
    epochs=Epochs
)


model.save(r'C:\Users\omdha\jupyter nb\mobilenet_v2.model')

C:\Users\omdha\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
91/91 [==============================] - ETA: 0s - loss: 0.2332 - accuracy: 0.9154

In [ ]:
predict=model.predict(test_X,batch_size=BS)
predict=np.argmax(predict,axis=1)
print(classification_report(test_Y.argmax(axis=1),predict,target_names=lb.classes_))